# Install Dependencies


In [28]:
!curl -Hx-api-key:"<INSERT SEMANTIC SCHOLAR API KEY HERE>" https://api.semanticscholar.org/

<!doctype html>
<html lang=en>
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to the target URL: <a href="https://www.semanticscholar.org/product/api">https://www.semanticscholar.org/product/api</a>. If not, click the link.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100   273  100   273    0     0    101      0  0:00:02  0:00:02 --:--:--   101


In [1]:
!pip install openai
!pip install python-decouple==3.8

In [1]:
import json
import openai
from decouple import config
import requests

In [21]:
OPEN_API_KEY = config('OPENAI_API_KEY')
openai.api_key = OPEN_API_KEY

# Prompt ChatGPT for prompts to query databases with

In [5]:
question = "How can we design a vehicle to travel from Sydney to USA in one hour?"

messages = []
messages.append({'role': 'user', 
'content': f"""I want to query an academic database to answer the question "{question}". \
Provide only the search query in a JSON without any additional text in the following format. \
{{"queries": ["query1", "query2"]}}"""})


response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
        )


In [6]:
messages

[{'role': 'user',
  'content': 'I want to query an academic database to answer the question "How can we design a vehicle to travel from Sydney to USA in one hour?". Provide only the search query in a JSON without any additional text in the following format. {"queries": ["query1", "query2"]}'}]

In [7]:
response

<OpenAIObject chat.completion id=chatcmpl-7jDmssOQPUegaduCzeAGq1pxWNEGw at 0x10858023fb0> JSON: {
  "id": "chatcmpl-7jDmssOQPUegaduCzeAGq1pxWNEGw",
  "object": "chat.completion",
  "created": 1691012114,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\"queries\": [\"Design principles for hypersonic vehicles\", \"Advancements in hypersonic travel\", \"Challenges in transcontinental hypersonic travel\", \"Propulsion systems for hypersonic vehicles\", \"Materials for high-speed aerospace vehicles\", \"Fuel efficiency in hypersonic vehicles\", \"Safety measures in hypersonic travel\"]}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 64,
    "completion_tokens": 62,
    "total_tokens": 126
  }
}

In [8]:
jsonstring = response['choices'][0]['message']['content']
jsonstring = "" + jsonstring

In [9]:
query_list = json.loads(jsonstring)
query_list['queries']

['Design principles for hypersonic vehicles',
 'Advancements in hypersonic travel',
 'Challenges in transcontinental hypersonic travel',
 'Propulsion systems for hypersonic vehicles',
 'Materials for high-speed aerospace vehicles',
 'Fuel efficiency in hypersonic vehicles',
 'Safety measures in hypersonic travel']

In [29]:
results = requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query=covid+vaccination&offset=100&limit=3')

In [30]:
json_results = json.loads(results.text)

In [31]:
json_results

{'total': 1170604,
 'offset': 100,
 'next': 103,
 'data': [{'paperId': '15241c09b59f6b1bbfa69867a29b554dff0cd04c',
   'title': 'Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage'},
  {'paperId': 'edd135e5dc5dbfe3f54c7d9f8be08706b06c779f',
   'title': 'Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review'},
  {'paperId': '73326621034f67beab8a6834345270b0379e47ec',
   'title': 'Myocarditis and Pericarditis After Vaccination for COVID-19.'}]}

In [32]:
for paper in json_results['data']:
    print(paper['title'])

Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage
Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review
Myocarditis and Pericarditis After Vaccination for COVID-19.


In [33]:
def stringToAPIformat(string1):
    string1 = string1.replace(" ", "+")
    return string1
assert stringToAPIformat("Hello there") == "Hello+there"

In [34]:
number_of_papers_to_retrieve = 3
data_to_retrieve = "title" # could be "abstract", "title"
# look into passing paper, getting summary 


for query in query_list['queries']:
    URLtoGET = f'https://api.semanticscholar.org/graph/v1/paper/search?query={stringToAPIformat(query)}&limit={number_of_papers_to_retrieve}&fields={data_to_retrieve}'
    GETresult = requests.get(URLtoGET)
    json_GETresult = json.loads(GETresult.text)
    print(f"Top {number_of_papers_to_retrieve} {data_to_retrieve}s for query: '{query}'")
    if json_GETresult['total'] == 0:
        print("No papers found \n")
        continue
    for paper in json_GETresult['data']:
        print("\t" + paper[data_to_retrieve])
    print("\n")
    

Top 3 titles for query: 'Design principles for hypersonic vehicles'
	Design and numerical simulation of a clamshell-shaped inlet cover for air-breathing hypersonic vehicles
	Design and numerical simulation of a clamshell-shaped inlet cover for air-breathing hypersonic vehicles
	Design of Dual-Mode Engine Flowpaths for Hypersonic Vehicles Using Reduced-Order Models


Top 3 titles for query: 'Advancements in hypersonic travel'
	Numerical study of a stepped aerospike design at various Mach numbers
	COMPRESSIBLE FLOW SIMULATIONS FOR SHOCK TUBE PERFORMANCE ANALYSIS


Top 3 titles for query: 'Challenges in transcontinental hypersonic travel'
No papers found 

Top 3 titles for query: 'Propulsion systems for hypersonic vehicles'
	Supersonic Combustion in Air-Breathing Propulsion Systems for Hypersonic Flight
	Supersonic Combustion in Air-Breathing Propulsion Systems for Hypersonic Flight
	HYPERSONIC PROPULSION SYSTEMS – A REVIEW OF DESIGN SOLUTIONS


Top 3 titles for query: 'Materials for high